In [ ]:
from ramanchada2.io.HSDS import visit_domain, filter_dataset, read_cha
import h5pyd
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

In [ ]:
from ramanchada2.spectrum import from_chada
import math


def load_dataset(parentdomain,domain,results={}):
    with h5pyd.File(domain) as f:
        wavelength = f["annotation_study"].attrs["wavelength"]
    spe = from_chada(domain,h5module=h5pyd)
    spe = spe - spe.moving_minimum(16)
    spe = spe.normalize()
    f = interp1d(spe.x, spe.y)
    x_min = min(spe.x)
    x_min = math.ceil(x_min) if x_min >= 0 else 0
    x_max = math.floor(max(spe.x))
    print(domain,len(spe.x),x_min,x_max)
    xnew = np.arange(x_min,x_max,1)
    ynew = f(xnew)

    results[domain] = (xnew,ynew,wavelength)

query_sample = "S1N"

#visit_domain(topdomain="/Round_Robin_1/",process_dataset=print_dataset) 
results = {}
visit_domain("/Round_Robin_1/",
        process_dataset=filter_dataset,kwargs={"process_file" : load_dataset,"sample": query_sample, 
                            "kwargs" : {"results" : results}})

In [ ]:

def ramanshift2wavelength(shift,laser):
    return 1/((1/laser)-shift/10**7)

domain_len = 2048
xnew0 = np.arange(0,domain_len,1)
xnew1 = np.arange(0,domain_len,1)
xnew1 = xnew1 + domain_len
for domain in results:
    x,y,wavelength = results[domain]
    wavelength = int(wavelength)
    index0 = np.zeros(domain_len)
    index1 = np.zeros(domain_len)
    xmin = min(x)
    xmax = max(x)
    #print(wavelength,min(x),max(x))
    xmax = xmax if xmax <=domain_len else domain_len
    xlen = len(x) if len(x) <= domain_len else domain_len
    index0[xmin:xmax] = y[0:(xmax-xmin)]
    #print(xmin,xmax,xmax-xmin)
    index0[0:16] = 0
    if len(x)>(xmax-xmin):
        #xmax = xmax if xmax <= (2*domain_len) else 2*domain_len
        #print(xmax-xmin,"more to come",min(x),max(x))
        #index1[xmax:] = y[(xmax-xmin):]
        pass
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    ax1.plot(ramanshift2wavelength(x,wavelength),y, 'o',label=domain)
    ax1.plot(ramanshift2wavelength(xnew0,wavelength),index0, '-', label=domain)
    ax1.set_xlabel("wavelength, nm")
    ax2.plot(x,y, 'o',label=domain)
    ax2.plot(xnew0,index0, '-', label=domain)    
    ax2.set_xlabel("Raman shift, cm-1")
    
    #plt.plot(xnew1,index1, 'x')
    plt.suptitle(domain)
    plt.show()